# 연습문제 3 - 양자 오류 수정 Quantum error correction

## 역사적 배경

쇼어 알고리즘(Shor's algorithm)은 양자 컴퓨터에 가치 있는 이용 사례를 주었지만 내재하는 양자역학의 노이즈때문에 이러한 알고리즘을 실행할 수 있는 하드웨어를 만드는 것은 힘든 일이었습니다. 1995년에 쇼어는 다른 획기적인 논문 (에러를 낮추기 위한 다수의 큐비트들을 넘어선 공유하는 앙자 정보 방안[1])을 발표했었습니다.

그 후 수십 년 동안 많은 진전과 발전이 있었습니다. 새로운 형태의 오류 수정(error correcting) 코드들이 발견되었고, 이를 중심으로 커다란 이론적 틀이 구축되었습니다. 1997년에 키타에프(Kitaev)가 제안한 surface code들이 유력 후보로 떠올랐고, 이후 기존 디자인의 많은 변형이 나타났었습니다. 그러나 아직 코드들을 양자 하드웨어의 구체적인 세부 사항에 맞춰 조정하는 데에 많은 진전과 발전이 필요합니다.[2]

이번 연습에서는 우리는 회로에 인위적인 에러들(artificial errors)이 삽입되는 경우를 고려할 것입니다. 이러한 추가적인 게이트들을 식별할 수 있도록 회로를 설계하는 것이 여러분의 과제입니다.

그런 다음에 실제 장치(real device)에 어떻게 회로를 구현할지 생각해야 합니다. 즉 여러분은 솔루션을 큐비트들의 레이아웃에 맞춰 조정해야 합니다. 솔루션은 얼마나 적은 entangling gates (가장 소음이 심한 게이트 유형)들이 사용되었는지에 따라 점수가 매겨집니다.

### 참고자료
1. Shor, Peter W. "Scheme for reducing decoherence in quantum computer memory." Physical review A 52.4 (1995): R2493.
1. Dennis, Eric, et al. "Topological quantum memory." Journal of Mathematical Physics 43.9 (2002): 4452-4505.

## 오류의 문제점들

오류는 일부 의사 작동(spurious operation)이 큐비트들에게 수행될 때 발생합니다. 이러한 결과들은 우리의 회로에서 문제들을 발생시킵니다. 실제 장치(real device)에서 실행할 때 생기는 이상한 결과들은 모두 이러한 오류들 때문일 수 있습니다.

많은 의사 작동(spurious operations)들이 일어날 수 있지만, 우리는 두 가지의 오류, 즉 bit flips와 phase flips만 있다고 가정할 수 있습니다.

Bit flips는 `x` 게이트와 같은 결과를 줍니다. 이들은 단일 큐비트의 $|0\rangle$ 상태를 $|1\rangle$로 뒤집고 그 반대도 마찬가지로 뒤집습니다. Phase flips는 `z` 게이트와 같은 결과를 나타내며 $-1$ 위상(phase)을 중첩(superposition)에 도입합니다. 간단히 말해서, 이들은 단일 큐비트의 $|+\rangle$ 상태를 $|-\rangle$로 뒤집고 그 반대로도 뒤집습니다.

우리가 모든 오류를 이 두 가지로만 생각할 수 있는 이유는 모든 오류는 일부 행렬로 표현될 수 있고 모든 행렬은 $X$ 와 $Z$ 행렬로 표현할 수 있기 때문입니다. 구체적으로 적절하게 선택된 값 $\alpha$, $\beta$, $\gamma$ 그리고 $\delta$ 에 대해 모든 단일 큐비트 행렬 $M$ 은 아래와 같습니다.

$$
M = \alpha I + \beta X + \gamma XZ + \delta Z,
$$

따라서 이 행렬을 $|\psi\rangle$ 상태의 어떤 단일 큐비트에 적용할 때마다 우리는 아래와 같이 얻을 수 있습니다.

$$
M |\psi\rangle = \alpha |\psi\rangle + \beta X |\psi\rangle + \gamma XZ |\psi\rangle + \delta Z |\psi\rangle.
$$

결과적 중첩은 원래 상태(original state), bit flip 오류만으로 만들어진 상태, phase flip 오류만으로 만들어진 상태, 두 가지 모두로 만들어진 상태로 구성됩니다. 만약 우리가 bit flip이나 phase flip이 일어났는지 측정하는 방법을 가지고 있다면, 그 상태는 단지 하나의 가능성으로 붕괴할 것입니다. 그리고 우리의 복잡한 오류는 간단한 bit flip이나 phase flip이 될 것입니다..

그러면 우리는 어떻게 bit flip이나 phase flip (또는 둘 다)를 감지할 수 있을까요. 일단 알게 되면 어떻게 해야 할까요? 이러한 질문에 답하는 것이 양자 오류 수정(quantum error correction)의 모든 것입니다.

## 간단한 예

대부분의 사람들이 첫 번째로 작성하는 양자 회로 중 하나는 얽힌 큐비트(entangled qubits) 쌍을 만드는 것입니다. 양자 오류 수정(quantum error correction)을 향한 이 여정에서도 우리는 같은 방식으로 시작할 것입니다.

In [ ]:
from qiskit import QuantumCircuit, Aer

# 얽혀진 쌍(entangled pair) 만들기
qc_init = QuantumCircuit(2)
qc_init.h(0)
qc_init.cx(0,1)

# 회로 그리기
display(qc_init.draw('mpl'))

# 출력 얻기
qc = qc_init.copy()
qc.measure_all()
job = Aer.get_backend('qasm_simulator').run(qc)
job.result().get_counts()

여기서 회로를 실행할 때 예상된 결과: `00`과 `11`이 동일한 확률로 발생하는 것을 볼 수 있습니다.

하지만 동일한 회로에 bit flip 오류가 삽입됐을 때는 어떠한 현상이 일어날까요.

In [ ]:
# bit flip 오류 만들기
qc_insert = QuantumCircuit(2)
qc_insert.x(0)

# 기존 회로에 추가하기
qc = qc_init.copy()
qc = qc.compose(qc_insert)

# 회로 그리기
display(qc.draw('mpl'))

# 출력 얻기
qc.measure_all()
job = Aer.get_backend('qasm_simulator').run(qc)
job.result().get_counts()

이제 결과가 `01`과 `10`로 바뀌었습니다. 두 비트 값은 항상 동일한 것에서 항상 같지 않는 것으로 바뀌었습니다. 이러한 방법으로 우리는 오류의 결과를 탐지합니다.

다른 탐지 방법으로 몇몇 게이트들을 추가로 사용해서 얽힘을 푸는 방법이 있습니다. 오류가 없다면 회로는 초기 $|00\rangle$ 상태로 돌아갑니다.

In [ ]:
# 얽힘 풀기
qc_syn = QuantumCircuit(2)
qc_syn.cx(0,1)
qc_syn.h(0)

# 오류 이후에 이것을 추가하기
qc = qc_init.copy()
qc = qc.compose(qc_syn)

# 회로 그리기
display(qc.draw('mpl'))

# 출력 얻기
qc.measure_all()
job = Aer.get_backend('qasm_simulator').run(qc)
job.result().get_counts()

하지만 큐비트 중 하나에 오류가 발생하면 어떻게 될까요? 다른 오류들을 삽입해서 확인해보세요.

지금까지 소개한 모든 구성 요소, 초기화 `qc_init`, `qc_insert`에 삽입된 오류, 최종 측정에서 정확한 답을 얻는 최종 `qc_syn`가 포함된 회로가 아래에 있습니다.

In [ ]:
# 오류 정의하기
qc_insert = QuantumCircuit(2)
qc_insert.x(0)

# 얽힘 풀기
qc_syn = QuantumCircuit(2)
qc_syn.cx(0,1)
qc_syn.h(0)

# 오류 이후에 이것을 추가하기
qc = qc_init.copy()
qc = qc.compose(qc_insert)
qc = qc.compose(qc_syn)

# 회로 그리기
display(qc.draw('mpl'))

# 출력 얻기
qc.measure_all()
job = Aer.get_backend('qasm_simulator').run(qc)
job.result().get_counts()

계산 결과를 살펴보면 오류의 원인을 정확하게 알 수 있다는 것을 알게 될 것입니다. bit filp과 phase flips 모두 감지할 수 있습니다. bit flip (`x(0)`나 `x(1)`을 삽입한 경우)이 있어야 왼쪽의 비트 값은 `1`이 됩니다 비슷하게 오른쪽 비트는 phase flip (`z(0)`나 `z(1)`가 삽입된)이 있다는 걸 나타냅니다.

bit flip과 phase flip을 감지하고 구별할 수 있는 이 기능은 매우 유용합니다. 하지만 이것으로는 충분하지 않습니다. 이것을 통해 우리는 *어디서* 오류가 생겼나가 아닌 *어떤 유형*의 오류가 일어났는지만을 알 수 있습니다. 더 자세한 정보가 없으면 우리의 연산에서 이러한 오류결과 들을 제거하는 것이 불가능합니다. 따라서 양자 오류 수정을 위해 우리는 더 크고 더 나은 무언가가 필요합니다.

이것의 여러분의 도전 과제입니다! 다음은 제출해야 하는 항목의 목록입니다. 여기 있는 모든 것은 다음에 나오는 예제를 통해 설명됩니다.

<div class="alert alert-block alert-success">

<b>목표</b>

두 큐비트에서 `x`와 `z` 오류들을 감지할 수 있는 회로들을 만드세요.
당신만의 솔루션을 생각해 만들어 낼 수 있습니다. 또는 아래에 제시된 거의 유효한 솔루션을 수정하셔서 완성하세요.

</div>

<div class="alert alert-block alert-danger">
<b>제출할 내용<b> 

* 아래와 같은 두 개의 회로를 제공해야 합니다:
    * `qc_init`: 원하는 초기 상태로 큐비트(이 중 최소 2개)를 준비합니다;
    * `qc_syn`: 큐비트들의 부분 집합(subset)을 측정합니다.

* 삽입될 인위적 오류(artificial errors) 두 개의 특정 큐비트에 있는 `x`와 `z` 게이트들입니다. 이 작업에 사용될 두 개의 큐비트를 선택해야 합니다 (목록 `error_qubits`으로 제공됨)

* 오류가 없는 단순한 경우를 포함해 삽입 가능한 총 16개의 오류 세트들이 있습니다. `qc_syn`의 측정 결과는 각각에 대해 고유한 비트 문자열을 출력해야 합니다. 만약 조건에 충족하지 않는다면  *'회로가 초기 레이아웃으로 작성되었는지 확인하십시오 (Please make sure the circuit is created to the initial layout)'*라는 오류 메시지가 표시됩니다.

* 채점자는 만들어진 회로를 백엔드 `ibmq_tokyo` (퇴역 기기)에서 컴파일 할 것입니다. 솔루션이 장치에 적합하다는 것을 보여주기 위해, transpilation에서는 `cx` 게이트들의 수를 변경하지 않아야 합니다. 만약 수가 변경되었다면, *'회로가 초기 레이아웃으로 생성되었는지 확인하십시오 (Please make sure the circuit is created to the initial layout.)'*라는 오류 메시지가 표시됩니다.
    
* transpilation을 유도하려면, 트랜스파일러에게 장치의 어떤 큐비트들이 회로의 어떤 큐비트로 사용해야 하는지 명시해야 합니다. 이 작업은 `initial_layout` 목록으로 수행됩니다.
    
* 아래에 제시된 예로부터 시작할 수 있으며, 몇 가지 변경으로 유효한 솔루션을 만들 수 있습니다.
</div>

## 더 좋은 예: the surface code

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, transpile

import qiskit.tools.jupyter
from qiskit.test.mock import FakeTokyo

이번 예에서는 코드 큐비트(code qubit)라고 불리는 5개의 큐비트를 사용할 것입니다. 이들을 추적하기 위해, 는 특별한 양자 레지스터를 정의해야 합니다.

In [ ]:
code = QuantumRegister(5,'code')

또한, 우리는 신드롬 큐비트(syndrome qubit)라고 불리는 4개의 큐비트를 추가로 가질 것입니다.

In [ ]:
syn = QuantumRegister(4,'syn')

마찬가지로 우리는 신드롬 큐비트를 측정할 때 사용되는 4개의 출력 비트에 대한 레지스터를 정의합니다.

In [ ]:
out = ClassicalRegister(4,'output')

우리는 코드 큐비트가 네 개의 삼각형 모서리를 형성하고 신드롬 큐비트가 각 삼각형의 내부에 배치되어 있는 것을 아래와 같이 생각합니다.

```
c0----------c1
| \   s0   / |
|   \    /   |
| s1  c2  s2 |
|   /    \   |
| /   s3   \ |
c3----------c4
```

각 삼각형마다 3개의 큐비트에 스태빌라이저 (stabilizer)을 연결합니다. 측면의 큐비트의 경우 스태빌라이저는 ZZZ입니다. 상단 및 하단의 경우에는 XXX입니다.

신드롬 측정 회로는 이러한 관측의 측정에 해당합니다. surface code 스태빌라이저에서도 이와 유사한 방식으로 수행됩니다 (사실, 이 코드는 surface code의 작은 버전입니다).


<div class="alert alert-block alert-danger">
 
<b>경고</b> 

transpilation에 방해가 될 수 있으므로 코드를 제출하기 전에 장벽을 제거해야 합니다. 여기서는 시각화를 위해 주어졌을 뿐입니다.
</div>

In [ ]:
qc_syn = QuantumCircuit(code,syn,out)


# 좌측 ZZZ
qc_syn.cx(code[0],syn[1])
qc_syn.cx(code[2],syn[1])
qc_syn.cx(code[3],syn[1])
qc_syn.barrier()

# 우측 ZZZ
qc_syn.cx(code[1],syn[2])
qc_syn.cx(code[2],syn[2])
qc_syn.cx(code[4],syn[2])
qc_syn.barrier()

# 상단 XXX
qc_syn.h(syn[0])
qc_syn.cx(syn[0],code[0])
qc_syn.cx(syn[0],code[1])
qc_syn.cx(syn[0],code[2])
qc_syn.h(syn[0])
qc_syn.barrier()

# 하단 XXX
qc_syn.h(syn[3])
qc_syn.cx(syn[3],code[2])
qc_syn.cx(syn[3],code[3])
qc_syn.cx(syn[3],code[4])
qc_syn.h(syn[3])
qc_syn.barrier()


# 예비 큐비트 측정
qc_syn.measure(syn,out)
qc_syn.draw('mpl')

초기화 회로는 신드롬 측정 결과가 확실하게 `0000`이 되도록 이러한 관측 가능성의 고유상태를 준비합니다.

In [ ]:
qc_init = QuantumCircuit(code,syn,out)

qc_init.h(syn[0])
qc_init.cx(syn[0],code[0])
qc_init.cx(syn[0],code[1])
qc_init.cx(syn[0],code[2])
qc_init.cx(code[2],syn[0])

qc_init.h(syn[3])
qc_init.cx(syn[3],code[2])
qc_init.cx(syn[3],code[3])
qc_init.cx(syn[3],code[4])
qc_init.cx(code[4],syn[3])

qc_init.barrier()
qc_init.draw('mpl')

이것이 사실인지 확인해 봅시다.

In [ ]:
qc = qc_init.compose(qc_syn)
display(qc.draw('mpl'))

job = Aer.get_backend('qasm_simulator').run(qc)
job.result().get_counts()

이제 우리의 두 개의 코드 큐비트에 `x`와 `z` 게이트들을 삽입할 수 있는 회로를 만듭시다. 이를 위해서 5개의 코드 큐비트 중 유효 조건에 필요한 2개의 코드 큐비트를 선택해야 합니다. 

이 코드에서 우리는 반대쪽 모서리를 선택합니다.

In [ ]:
error_qubits = [0,4]

여기서 0과 4는 다음 목록에서 큐비트의 위치를 가리킨다, 따라서 큐비트는 `code[0]`와 `code[4]`가 된다.

In [ ]:
qc.qubits

코드가 원하는 대로 작동하는지 확인하기 위해 다음 기능을 사용해 인공 오류(artificial errors)를 삽입하는 회로를 만들 수 있습니다. 여기서 추가하려는 오류는 `errors`에 `error_qubits[0]`의 `x`에 대한 `x0`같이 간단한 텍스트 문자열로 나열됩니다.

In [ ]:
def insert(errors,error_qubits,code,syn,out):

    qc_insert = QuantumCircuit(code,syn,out)

    if 'x0' in errors:
        qc_insert.x(error_qubits[0])
    if 'x1' in errors:
        qc_insert.x(error_qubits[1])
    if 'z0' in errors:
        qc_insert.z(error_qubits[0])
    if 'z1' in errors:
        qc_insert.z(error_qubits[1])
        
    return qc_insert

16개의 모든 가능성보다는 하나의 오류가 삽입되는 네 가지 경우를 살펴봅시다.

In [ ]:
for error in ['x0','x1','z0','z1']:
    
    qc = qc_init.compose(insert([error],error_qubits,code,syn,out)).compose(qc_syn)
    job = Aer.get_backend('qasm_simulator').run(qc)
    
    print('\nFor error '+error+':')
    counts = job.result().get_counts()
    for output in counts:
        print('Output was',output,'for',counts[output],'shots.')

가장 왼쪽이 `error_qubits[1]`에서 `z`를 감지한 후, 다음 감지가 `error_qubits[1]`에서의 `x`를 감지하고 기타등등 처럼 특정한 오류가 발생할 때 출력된 각 비트가 `1`이라는걸 알 수 있습니다.

<div class="alert alert-block alert-danger">
 
<b>주의</b> 

이번 예제에서는 정확한 출력 양식이 중요합니다. 아래의 내용을 지켜주세요:

1. 가장 왼쪽 출력은 `code[1]`의 `z`를 나타냅니다.
2. 왼쪽에서의 두 번째 출력은 `code[1]`의 `x`를 나타냅니다.
3. 왼쪽에서의 세 번째 출력은 `code[0]`의 `x`를 나타냅니다.
4. 가장 오른쪽 출력은 `code[0]`의 `z`를 나타냅니다.  
</div>

더 많은 오류가 회로에 영향을 끼치면 어떤 오류가 발생했는지 명확하게 알 수 없게 됩니다. 하지만 신드롬 판독을 지속적으로 반복해서 더 많은 결과와 디코딩 프로세스를 통한 데이터를 분석함으로써 오류를 판단하고 수정하기에는 충분합니다.

이러한 고려사항들은 우리가 이번 챌린지에서 다루게 되는 범위를 벗어납니다. 대신, 우리는 더욱 간단하지만 적고 간편한 중요한 것에 초점을 맞출 겁니다. 여러분이 가지고 있는 오류가 더 적고, 더 단순할수록, 여러분의 오류 수정도 더 좋아질 것입니다. 이 목적을 달성하기 위해 오류 수정 절차는 사용하는 기기에 맞춰줘야 합니다.

이번 챌린지에서는 우리는 `ibmq_tokyo`기기를 사용하게 될 것입니다. 비록 실제 버전은 얼마 전에 은퇴했지만, 여전히 하나의 모의의 백엔드로 남아있습니다.

In [ ]:
# 여기에 주어진 백엔드를 사용하세요
backend = FakeTokyo()
backend

위에서 사용한 회로가 레이아웃에 몇 개의 2 큐비트 게이트를 갖고 있는지 간단하게 확인해 봅시다.

In [ ]:
qc = qc_init.compose(qc_syn)
qc = transpile(qc, basis_gates=['u','cx'])
qc.num_nonlocal_gates()

`ibmq_tokyo` 백엔드로 트렌스파일을 한다면, 2큐빗 게이트를 추가하는 비용으로 리매핑(remapping)이 이루어져야 합니다.

In [ ]:
qc1 = transpile(qc,backend,basis_gates=['u','cx'], optimization_level=3)
qc1.num_nonlocal_gates()

장치에 있는 어떤 큐비트를 코드에 있는 큐비트로 사용하는 것이 좋을지를 찾아봄으로써 우리는 이것을 어느 정도 제어할 수 있습니다. `qc_syn`에 있는 2큐비트 게이트로 연결되어야 하는 코드의 큐비트를 살펴본다면 다음과 같은 연결 그래프를 찾아볼 수 있습니다.


```
c0....s0....c1
:      :     :        
:      :     :
s1....c2....s2
:      :     :
:      :     :
c3....s3....c4
```

`ibmq_tokyo`에 있는 어떠한 큐비트 세트들은 이것을 제공할 수 없지만 0,1,2,5,6,7,10,11,12와 같은 특정 세트들은 유사하게 근접합니다. 따라서 `initial_layout`을 이용해 트랜스파일러에게 이러한 것을 사용하도록 설정할 수 있습니다.


In [ ]:
initial_layout = [0,2,6,10,12,1,5,7,11]

이러한 것은 트랜스파일러에게 기기 중에 어떤 큐비트를 회로에 있는 큐비트들을 위해 사용할지 알려줍니다. (이는 `qc.qubits`에 나열된 순서대로 진행됨). 그래서 목록의 처음 5개의 항목은 회로에서 코드 큐비트로 사용하는지 알려주고 다음 3개의 항목은 비슷하게 회로에서 신드롬 큐비트로 사용되는지 알려줍니다. 따라서 우리는 기기에 있는 큐비트 0을 `code[0]` 와 같이, 큐비트 2를 `code[1]` 와 같이, 등으로 사용하게 됩니다.

자 이제 이걸 트랜스파일러에 사용해봅시다.

In [ ]:
qc2 = transpile(qc,backend,initial_layout=initial_layout, basis_gates=['u','cx'], optimization_level=3)
qc2.num_nonlocal_gates()

트랜스파일러는 무작위로 진행(random process)되지만, 일반적으로 초기 레이아웃이 제공되지 않을 때보다 2큐비트 게이트를 덜 사용하게 됩니다 (트랜스파일러가 무작위 진행인 걸 보려면 트랜스파일러 코드를 여러 번 다시 실행해야 합니다).

그럼에도 불구하고, 적절하게 설계된 오류 수정 체계는 리매핑(remapping)이 전혀 필요하지 않아야 합니다. 장치에 맞게 정확하게 작성되어야 하며 2 큐비트 게이트의 수는 확실히 일정하게 유지되어야 합니다. 이것은 솔루션이 유효하기 위한 조건입니다. 따라서 `initial_layout`은 제공해야 할 뿐만 아니라 해당 레이아웃에 맞게 회로를 설계해야 합니다.

하지만 그 부분은 여러분에게 맡기겠습니다!

In [ ]:
# 다음과 같은 코드를 사용하여 답을 확인하세요.
from qc_grader import grade_ex3
grade_ex3(qc_init,qc_syn,error_qubits,initial_layout)

In [ ]:
# 답을 제출하세요. 언제나 다시 제출할 수 있습니다.
from qc_grader import submit_ex3
submit_ex3(qc_init,qc_syn,error_qubits,initial_layout)

## 추가 정보

**출제자:** James Wootton, Rahul Pratap Singh

**번역:** 최인호 (qiskit slack @Inho Choi)

**버젼:** 1.0.0